In [20]:
import sys
import regex

In [21]:
DEBUG = True

In [22]:
def read_file(file,process_line):
    with open(file,encoding='UTF-8') as f:
        for line in f:
            process_line(line)

In [23]:
def process_line(line):
    expression = '\[\[([\p{L}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pc}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})=>]+)\|?([\p{L}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pc}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})=>]+)?\]\]'
    matches = regex.findall(expression,line)
    if len(matches) > 0:
        for match in matches:
            print(f"{match}, ")
        print('\n')

In [24]:
file_name = '../data/test.xml' if DEBUG else '../data/cswiki-latest-pages-articles.xml'

In [25]:
read_file(file_name, process_line)

('Diskuse k Wikipedii:WikiProjekt Kvalita/Kritéria#Dobrý_nebo_Nejlepší_portál?', 'dle diskuse'), 


('Wikipedie:Průvodce', 'Vítejte ve&amp;nbsp;Wikipedii'), 


('encyklopedie', 'encyklopedii'), 
('Nápověda:Úvod', 'upravovat každý'), 
('Česká Wikipedie', ''), 
('Speciální:Statistika', '{{POČETČLÁNKŮ}}'), 
('Speciální:Nové stránky', 'článků'), 


('Wikipedie:Seznam jazyků Wikipedie', 'Jiné jazyky'), 
('Wikipedie:Velvyslanectví', 'Embassy'), 


('Wikipedie:Obrázek týdne', 'Obrázek týdne'), 


('commons:Commons:Featured pictures/cs', 'Nejlepší obrázky'), 
('Wikipedie:Obrázek týdne/{{MÍSTNÍROK}}', 'Další obrázky týdne…'), 


('Wikipedie:Článek týdne', 'Článek týdne'), 


('Wikipedie:Nejlepší články', 'Nejlepší články'), 
('Wikipedie:Dobré články', 'Dobré články'), 
('Wikipedie:Článek týdne/{{MÍSTNÍROK}}', 'Další články týdne…'), 


('Wikipedie:Zajímavosti', 'Víte, že…'), 


('Wikipedie:Zajímavosti/{{MÍSTNÍROK}}', 'Další zajímavosti…'), 


(':Kategorie:Hlavní kategorie', 'Listování kategorie